# **Transformers and Natural Language Processing for Knowledge Tracing:**

This notebook will contain an implementation of applying TF-IDF to problems textual descriptions, after necessary data preprocessing and of using consine similarity on TF-IDF values to calculate similarity between problems and predict result for following problem. 

# Import necessary libraries:

In [1]:
#import needed libraries 
from urllib.request import urlretrieve
import zipfile, os
import time, sys, copy
import pandas as pd
import scipy.sparse as sps
import numpy as np
from collections import defaultdict
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import hunspell
import psutil
import gc
from sklearn.metrics import roc_auc_score, accuracy_score
import smart_open
import six
import gensim
from gensim.models import Word2Vec
from time import time  # To time our operations
from gensim.models import KeyedVectors

ModuleNotFoundError: No module named 'sklearn'

# Clone github repositories from previous works

In [ ]:
#clone repositeries
!git clone https://github.com/shalini1194/RKT
#!git clone https://github.com/lyf-1/PEBG.git
!git clone https://github.com/jhljx/GKT.git   
!git clone https://github.com/Shivanandmn/Knowledge-Tracing-SAINT.git
!git clone https://github.com/MaurizioFD/RecSys_Course_AT_PoliMi
#copy repositories in working directory
!cp -r ./RKT/* ./
!cp -r ./RecSys_Course_AT_PoliMi/* ./

!pip install hunspell
"""!pip install -r ./RecSys_Course_AT_PoliMi/requirements.txt
!python ./RecSys_Course_AT_PoliMi/run_compile_all_cython.py
"""

In [ ]:
from Base.Similarity.Compute_Similarity import Compute_Similarity

# Import datasets

Assistments 2012/13

In [5]:
from import_assistments_files import import_data as imp
assistments_problems = imp("Assistments/problem_bodies/ASSISTments2012DataSet-ProblemBodies.csv")
print(assistments_problems[0])

FileNotFoundError: [Errno 2] No such file or directory: './Knowledge_Tracing/data/Assistments/problem_bodies/ASSISTments2012DataSet-ProblemBodies.csv'

Junyi

In [ ]:
input_folder = '../input/'
# Junyi with problems textual descriptions 
# Data folder + File name must be the path to dataset. 
file_name = 'junyi-dataset/junyi_question_text.txt'
df = pd.read_csv(os.path.join(input_folder, file_name),low_memory=False, sep = '#')

Peking Online Judge (POJ)

In [ ]:
input_folder = '../input/'
# POJ with problems textual descriptions 
# Data folder + File name must be the path to dataset. 
file_name = 'poj-dataset/poj_question_text.txt'
df = pd.read_csv(os.path.join(input_folder, file_name),low_memory=False, sep = '\n', names=["data"])

# Texts preprocessing and cleaning:

In [ ]:
eng_dict = hunspell.HunSpell('/usr/share/hunspell/en_US.dic', '/usr/share/hunspell/en_US.aff')

problem_ids, assistment_ids, bodies = df['problem_id'], df['assistment_id'], df['body']
texts=[]
texts_no_numbers=[]
texts_only_existing_words=[]

nltk.download('stopwords')
for body in bodies:
    words_set = set({})
    words_set_no_numbers = set({})
    words_set_only_existing_words = set({})
    text = str(body).replace(' ', '#').replace('/', '#slash#').replace('<', '#lessthan#').replace('>', '#morethan#').replace(",", "#comma#").replace(";", "#semicolon#").replace(".", "#point#").replace("?", "#questionmark#").replace("!", "exclamationpoint").replace("=", "#equal#").replace("\\", "#slash#").replace("%", "#percentage#").replace("\\t", "#").replace("\\n", "#").replace("\t", "#").replace("\n", "#").replace('\"', "#quotationmark#").replace("(", "#openroundbracket#").replace(")", "#closeroundbracket#").replace("[", "#opensquarebracket#").replace("]", "#closesquarebracket#").replace("_", "#underscore#").replace("&", "#ampersand#").replace("}", "#closebrace#").replace("{", "#openbrace#").replace("+", "#plus#").replace("-","#minus#").replace("*", "#multiplication#").replace("€","#euros#").replace("$","#dollar#").replace("^","#powerof#exponent#")
    text = str(text).split('#')
    for i in range(0, len(text)):
        text[i].lower()
        if eng_dict.spell(text[i]):
            words_set_only_existing_words.add(text[i])
        """if len(text[i])<20:
            words_set.add(text[i])
            if not (text[i].isdecimal()):
                result = ''.join(el for el in text[i] if not el.isdigit())
                words_set_no_numbers.add(result)"""
    """text = list(words_set)
    text_no_numbers = list(words_set_no_numbers)"""
    text_only_existing_words = list(words_set_only_existing_words)
    for i in stopwords.words('english'):
        i = i.lower()
        """if text.count(i)>0:
            text.remove(i)
        if text_no_numbers.count(i)>0:
            text_no_numbers.remove(i)"""
        if text_only_existing_words.count(i)>0:
            text_only_existing_words.remove(i)
            
    """texts.append(text)
    texts_no_numbers.append(text_no_numbers)"""
    texts_only_existing_words.append(text_only_existing_words)



In [ ]:
eng_dict = hunspell.HunSpell('/usr/share/hunspell/en_US.dic', '/usr/share/hunspell/en_US.aff')

problem_names, questions, question_descriptions = df['question_name'], df['chinese_question'], df['chinese_question_desc']
texts_only_existing_words=[]
problem_ids = []
nltk.download('stopwords')
problem_ids = range(0, len(questions))
for index in range(0, len(questions)):
    words_set = set({})
    text = str(questions[index]).replace(' ', '#').replace('/', '#slash#').replace('<', '#lessthan#').replace('>', '#morethan#').replace(",", "#comma#").replace(";", "#semicolon#").replace(".", "#point#").replace("?", "#questionmark#").replace("!", "exclamationpoint").replace("=", "#equal#").replace("\\", "#slash#").replace("%", "#percentage#").replace("\\t", "#").replace("\\n", "#").replace("\t", "#").replace("\n", "#").replace('\"', "#quotationmark#").replace("(", "#openroundbracket#").replace(")", "#closeroundbracket#").replace("[", "#opensquarebracket#").replace("]", "#closesquarebracket#").replace("_", "#underscore#").replace("&", "#ampersand#").replace("}", "#closebrace#").replace("{", "#openbrace#").replace("+", "#plus#").replace("-","#minus#").replace("*", "#multiplication#").replace("€","#euros#").replace("$","#dollar#").replace("^","#powerof#exponent#")
    text = str(text).split('#')
    text_desc = str(question_descriptions[index]).replace(' ', '#').replace('/', '#slash#').replace('<', '#lessthan#').replace('>', '#morethan#').replace(",", "#comma#").replace(";", "#semicolon#").replace(".", "#point#").replace("?", "#questionmark#").replace("!", "exclamationpoint").replace("=", "#equal#").replace("\\", "#slash#").replace("%", "#percentage#").replace("\\t", "#").replace("\\n", "#").replace("\t", "#").replace("\n", "#").replace('\"', "#quotationmark#").replace("(", "#openroundbracket#").replace(")", "#closeroundbracket#").replace("[", "#opensquarebracket#").replace("]", "#closesquarebracket#").replace("_", "#underscore#").replace("&", "#ampersand#").replace("}", "#closebrace#").replace("{", "#openbrace#").replace("+", "#plus#").replace("-","#minus#").replace("*", "#multiplication#").replace("€","#euros#").replace("$","#dollar#").replace("^","#powerof#exponent#")
    text_desc = str(text_desc).split('#')
    text = text
    text = list(set(text) | set(text_desc))
    for i in range(0, len(text)):
        text[i].lower()
        if eng_dict.spell(text[i]):
            words_set.add(text[i])
    text_only_existing_words = list(words_set)
    for i in stopwords.words('english'):
        i = i.lower()
        if text_only_existing_words.count(i)>0:
            text_only_existing_words.remove(i)
    if text_only_existing_words.count('TIMEOUT')>0:
        text_only_existing_words.remove('TIMEOUT')
    if text_only_existing_words.count('ISSUE')>0:
        text_only_existing_words.remove('ISSUE')
    if text_only_existing_words.count('underscore')>0:
        text_only_existing_words.remove('underscore')
    texts_only_existing_words.append(text_only_existing_words)
print(texts_only_existing_words)

In [ ]:
eng_dict = hunspell.HunSpell('/usr/share/hunspell/en_US.dic', '/usr/share/hunspell/en_US.aff')
number_to_index =dict({})
questions = []
index = 0
questions.append([])
for row in df['data']:
    if '#' in row:
        array = row.split('#')
        if array[0].isdigit():
            questions.append([])
            index = index +1
            number = int(array[0])
            number_to_index[number] = index
            questions[index] = array[1]
            
    else:
        new = str(questions[index])+str(row)
        questions[index] = new
#print(phrases)
texts_only_existing_words=[]

nltk.download('stopwords')
for index in range(0, len(questions)):
    words_set = set({})
    text = str(questions[index]).replace(' ', '#').replace('/', '#slash#').replace('<', '#lessthan#').replace('>', '#morethan#').replace(",", "#comma#").replace(";", "#semicolon#").replace(".", "#point#").replace("?", "#questionmark#").replace("!", "exclamationpoint").replace("=", "#equal#").replace("\\", "#slash#").replace("%", "#percentage#").replace("\\t", "#").replace("\\n", "#").replace("\t", "#").replace("\n", "#").replace('\"', "#quotationmark#").replace("(", "#openroundbracket#").replace(")", "#closeroundbracket#").replace("[", "#opensquarebracket#").replace("]", "#closesquarebracket#").replace("_", "#underscore#").replace("&", "#ampersand#").replace("}", "#closebrace#").replace("{", "#openbrace#").replace("+", "#plus#").replace("-","#minus#").replace("*", "#multiplication#").replace("€","#euros#").replace("$","#dollar#").replace("^","#powerof#exponent#")
    text = str(text).split('#')
    for i in range(0, len(text)):
        text[i].lower()
        if eng_dict.spell(text[i]):
            words_set.add(text[i])
    text_only_existing_words = list(words_set)
    for i in stopwords.words('english'):
        i = i.lower()
        if text_only_existing_words.count(i)>0:
            text_only_existing_words.remove(i)
    if text_only_existing_words.count('TIMEOUT')>0:
        text_only_existing_words.remove('TIMEOUT')
    if text_only_existing_words.count('ISSUE')>0:
        text_only_existing_words.remove('ISSUE')
    if text_only_existing_words.count('underscore')>0:
        text_only_existing_words.remove('underscore')
    texts_only_existing_words.append(text_only_existing_words)
problem_ids = number_to_index.keys()


# WORD2VEC using Genism:

In [ ]:
w2v_model = Word2Vec(min_count=2,
                     window=5,
                     vector_size=300,
                     workers=3,
                     sg = 1)
t = time()

w2v_model.build_vocab(texts_only_existing_words, progress_per=100)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))
print(w2v_model)

t = time()

w2v_model.train(texts_only_existing_words, total_examples=w2v_model.corpus_count, epochs=10, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))


w2v_model.save("word2vec.model")
word_vectors = w2v_model.wv
word_vectors.save("word2vec.wordvectors")


# Load word2vec model:

In [ ]:
# Assistments 2012/13 load word2vec saved vectors (not anymore trainable) 
#wv = KeyedVectors.load_word2vec_format("word2vec.model")

# Assistments 2012/13 load word2vec saved model (trainable) 
w2v_model = Word2Vec.load("word2vec.model")
word_vectors = w2v_model.wv
similarity_matrix = np.dot(word_vectors.vectors, word_vectors.vectors.T)
print(word_vectors.vectors.shape)

In [ ]:
# junyi:
# problem_ids = problem_names

problem_to_text = dict({})

vocabulary = word_vectors.key_to_index
k = 0
print(len(problem_ids))
for (problem_id, t) in list(zip(*(problem_ids, texts_only_existing_words))):
    problem_words = []
    for word in t:
        if word in vocabulary:
            problem_words.append(word)
    problem_to_text[k] = problem_words
    k+=1
print(k)
pro_words = []

# TSNE visualization

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
 
import seaborn as sns
sns.set_style("darkgrid")

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

def tsnescatterplot(model, word, list_names):
    """ Plot in seaborn the results from the t-SNE dimensionality reduction algorithm of the vectors of a query word,
    its list of most similar words, and a list of words.
    """
    arrays = np.empty((0, 300), dtype='f')
    word_labels = [word]
    color_list  = ['red']

    # adds the vector of the query word
    arrays = np.append(arrays, model.wv.__getitem__([word]), axis=0)
    
    # gets list of most similar words
    close_words = model.wv.most_similar([word])
    
    # adds the vector for each of the closest words to the array
    for wrd_score in close_words:
        wrd_vector = model.wv.__getitem__([wrd_score[0]])
        word_labels.append(wrd_score[0])
        color_list.append('blue')
        arrays = np.append(arrays, wrd_vector, axis=0)
    
    # adds the vector for each of the words from list_names to the array
    for wrd in list_names:
        wrd_vector = model.wv.__getitem__([wrd])
        word_labels.append(wrd)
        color_list.append('green')
        arrays = np.append(arrays, wrd_vector, axis=0)
        
    # Reduces the dimensionality from 300 to 50 dimensions with PCA
    reduc = PCA(n_components=20).fit_transform(arrays)
    
    # Finds t-SNE coordinates for 2 dimensions
    np.set_printoptions(suppress=True)
    
    Y = TSNE(n_components=2, random_state=0, perplexity=15).fit_transform(reduc)
    
     # Sets everything up to plot
    df = pd.DataFrame({'x': [x for x in Y[:, 0]],
                       'y': [y for y in Y[:, 1]],
                       'words': word_labels,
                       'color': color_list})
    
    fig, _ = plt.subplots()
    fig.set_size_inches(9, 9)
    
    # Basic plot
    p1 = sns.regplot(data=df,
                     x="x",
                     y="y",
                     fit_reg=False,
                     marker="o",
                     scatter_kws={'s': 40,
                                  'facecolors': df['color']
                                 }
                    )
    
    # Adds annotations one by one with a loop
    for line in range(0, df.shape[0]):
         p1.text(df["x"][line],
                 df['y'][line],
                 '  ' + df["words"][line].title(),
                 horizontalalignment='left',
                 verticalalignment='bottom', size='medium',
                 color=df['color'][line],
                 weight='normal'
                ).set_size(15)

    
    plt.xlim(Y[:, 0].min()-50, Y[:, 0].max()+50)
    plt.ylim(Y[:, 1].min()-50, Y[:, 1].max()+50)
            
    plt.title('t-SNE visualization for {}'.format(word.title()))
    
tsnescatterplot(w2v_model, 'power', [i[0] for i in model.wv.most_similar(negative=["power"])])

# Calculate TF-IDF using libraries from scikit:

In [ ]:
def identity_tokenizer(text):
    return text
tfidf_vectorizer_existing_words_only = []
tfidf_vectorizer_existing_words_only = TfidfVectorizer(
    analyzer='word',
    tokenizer=identity_tokenizer,
    preprocessor=identity_tokenizer,
    token_pattern=None,
    use_idf = True)
tfidf_vectorizer_vectors_existing_words_only = tfidf_vectorizer_existing_words_only.fit_transform(texts_only_existing_words)
df_tf_idf_existing_words_only = pd.DataFrame.sparse.from_spmatrix(tfidf_vectorizer_vectors_existing_words_only)
print(df_tf_idf_existing_words_only.shape)

"""tfIdfVectorizer=TfidfVectorizer(use_idf=True, stop_words= 'english')
tfIdf = tfIdfVectorizer.fit_transform(texts_no_split)
df_tf_idf = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
df_tf_idf = df_tf_idf.sort_values('TF-IDF', ascending=False)
#df_tf_idf = df_tf_idf[df_tf_idf>0 and (not math.isnan(df_tf_idf))]
print (df_tf_idf.head(50))"""
#Now we change the name of the datasets available:
sparse_tf_idf = tfidf_vectorizer_vectors_existing_words_only
dataframe_tf_idf = df_tf_idf_existing_words_only

words_unique = tfidf_vectorizer_existing_words_only.get_feature_names()
#Save sparse matrix in current directory
data_folder = './'

sps.save_npz(os.path.join(data_folder, 'pro_words_existing_words_only.npz'), sparse_tf_idf)
#sps.save_npz(os.path.join(data_folder, 'pro_words_removed_digits.npz'), tfidf_vectorizer_vectors_no_numbers)

words_dict = dict({})
for i in range(0, len(words_unique)):
    words_dict[str(i)] = words_unique[i]
print(sparse_tf_idf)

def write_txt(file, data):
    with open(file, 'w') as f:
        for dd in data:
            f.write(str(dd)+'\n')
                    
write_txt(os.path.join(data_folder, 'words_set.txt'), words_unique)

shape of pro_word matrix before escaping any special character: (179950, 82084)

In the end I considered that escaping a symbol is better than removing, so I escape all the symbols, including { } * + - _ too, infact now symbols and words are cleaner, despite number has remained the same:    (179950, 62198)

Eventually we can consider removing words with digits to reduce number to: (179950, 44642)   --> 18000 less

A good alternative is to use Hunspell library to check if the word exist, removing not existing ones. In this cas we reduce number of words to: (179950, 27742)   --> 17000 less  --> very good!!

If we consider only existing words and remove digits too we obtain:

# Available dataset with TF-IDF values:

In [ ]:
pro_num = dataframe_tf_idf.shape[0]
words_num = dataframe_tf_idf.shape[1]
#print(words_unique) 
dataframe_tf_idf    #dense pandas dataframe
sparse_tf_idf       #sparse matrix


# Calculate cosine similarity between questions from TF-IDF dataset

In [ ]:
shrink = 10
topK = 100
normalize = True
similarity = "cosine"
similarity_matrix = Compute_Similarity(sparse_tf_idf.T, shrink=shrink, topK=topK, normalize=normalize, similarity = similarity).compute_similarity()
print(similarity_matrix[0])

sps.save_npz(os.path.join(data_folder, 'TF_IDF_pro_pro.npz'), similarity_matrix)



In [ ]:
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
def get_URM(path):
    process = psutil.Process(os.getpid())
    gc.enable()
    data = pd.read_csv(path, low_memory=False, encoding="ISO-8859-1", dtype={"user_id": int, "problem_id": int, "correct": float})
    user_list = data.user_id.to_list()
    problem_list = data.problem_id.to_list()
    y_list = data.correct.to_list()
    for n, i in enumerate(y_list):
        if i == 0.0:
            y_list[n] = -1.0
    del data
    print(process.memory_info().rss)
    URM_all = sps.coo_matrix((y_list, (user_list, problem_list)))
    URM_all = URM_all.tocsr()
    del y_list
    del user_list
    del problem_list
    return URM_all
data_path = '../input/skillbuilder-data-2009-2010/2012-2013-data-with-predictions-4-final.csv'
URM = get_URM(data_path)
print(URM)

In [ ]:
similarity_matrix = sps.load_npz('../input/assesments-12-13-precessed-data/TF_IDF_pro_pro.npz')

In [ ]:
sparse_cb_similarity_matrix = sps.load_npz('../input/assesments-12-13-precessed-data/pro_pro_.npz')
sparse_cb_similarity_matrix2 = sps.load_npz('../input/assesments-12-13-precessed-data/pro_pro_existing_words_only.npz')

#CBF.save_model(folder_path='./')


# **Evaluation of TF-IDF + cosine similarity**

In [ ]:
def _compute_problem_score_from_pro_pro_matrix(sparse_similarity_matrix, user_profile_array, correct, target_problem):
        """
        
        """
        """print(target_problem)
        print(sparse_similarity_matrix.tocsr().getrow(target_problem).todense()[0])"""
        item_scores = sparse_similarity_matrix.tocsr()[user_profile_array, :].dot(sparse_similarity_matrix.tocsr().getrow(target_problem).transpose())
        item_scores = item_scores.transpose().todense().dot(correct)
        if item_scores == 0.0:
            return -10.0
        return item_scores



In [ ]:
# WORD2VEC:
def _compute_problem_score_from_pro_pro_matrix(word_vectors, user_profile_array, correct, target_problem):
        """
        
        """
        item_scores = []
        if target_problem in problem_to_text:
            for problem_id in user_profile_array:
                if problem_id in problem_to_text and len(problem_to_text[problem_id])>0:
                    similarity = word_vectors.n_similarity( problem_to_text[problem_id], problem_to_text[target_problem] )
                else:
                    similarity = 0.0
                item_scores.append(similarity)
            item_scores = np.array(item_scores).transpose().dot(correct)
            return item_scores
        return -10.0

In [ ]:
data = np.load('../input/assesments-12-13-precessed-data/2012-2013-data-with-predictions-4-final.csv.npz')
y, problems, real_lens = data['y'], data['problem'], data['real_len']
pro_num = data['problem_num']
corrects = np.where(y==-1.0, -100.0, y)
corrects = np.where(corrects==0.0, -1.0, corrects)
corrects = np.where(corrects ==-100.0, 0.0, corrects)



In [ ]:
data = pd.read_csv('../input/junyi-dataset/junyi.csv', sep = '\n', names =['data'])
data = data['data']
print(data)
index = range(0, len(data)//4)
real_len_index = [el*4 for el in index]
real_lens = [int(data[x]) for x in real_len_index]
problem_index = [el*4+1 for el in index]
problem_data = [data[x].split(',') for x in problem_index]
corrects_index = [el*4+2 for el in index]
corrects_data = [data[x].split(',') for x in corrects_index]
labels =[]
predictions_w2v =[]
problems = []
corrects = []
for problem, correct, real_len in list(zip(*(problem_data, corrects_data, real_lens))):
    correct2 = [float(x) for x in correct]
    problem2 = [int(x) for x in problem]
    problem = problem2
    correct = np.where(correct2==-1.0, -100.0, correct2)
    correct = np.where(correct==0.0, -1.0, correct)
    correct = np.where(correct ==-100.0, 0.0, correct)
    problems.append(problem)
    corrects.append(correct)
pro_num = len(problem)



In [ ]:
data = pd.read_csv('../input/poj-dataset/poj_log.csv')
users = set(data['User'])
print(len(users))
real_lens = []
problems = []
corrects = []
for user, problem in data.groupby("User"):
    correct_answer = problem['Result']
    correct = []
    problem_df = problem["Problem"]
    problem_list = []
    k = 0
    for p, c in list(zip(*(problem_df, correct_answer))):
        if p in number_to_index:
            problem_list.append(number_to_index[p])
            if c == "Accepted":
                correct.append(1.0)
            else:
                correct.append(-1.0)
            k+=1
    if k>1:
        real_lens.append(k)
        problems.append(problem_list)
        corrects.append(correct)
print(problems[1])
print(corrects[1])
print(real_lens[1])

In [ ]:
problems_set_with_results = set([])
for problem, correct, real_len in list(zip(*(problems, corrects, real_lens))):
    # TF-IDF:
    problems_set_with_results = problems_set_with_results.union(set(problem))
problems_set_with_text = set(problem_ids)
print(problems_set_with_text)
print(problems_set_with_results)
problems_set_text_and_results = problems_set_with_text.intersection(problems_set_with_results)
print(len(problems_set_text_and_results))

In [ ]:
predictions_TF_IDF =[]
predictions_w2v = []
random_predictions =[]
labels = []
i = 0
max_prob = 0
print(np.amax(problems))
print(len(problems))
for problem, correct, real_len in list(zip(*(problems, corrects, real_lens))):
    # TF-IDF:
    problem2 = []
    correct2 = []
    target_problem = problem[real_len-1]
    for p in range(0, len(problem)):
        if p in problems_set_text_and_results:
            problem2.append(problem[p])
            correct2.append(correct[p]) 
    real_len2 = len(problem2)
    if target_problem in problems_set_text_and_results:
        prediction_TF_IDF = _compute_problem_score_from_pro_pro_matrix(similarity_matrix, problem2[0:real_len2-1], correct2[0:real_len2-1], problem2[real_len2-1])  
        # WORD2VEC:
        #prediction_w2v = _compute_problem_score_from_pro_pro_matrix(word_vectors, problem[0:real_len-1], correct[0:real_len-1], problem[real_len-1]) 
        if prediction_TF_IDF >-10.0:
            if prediction_TF_IDF >= 0:
                predictions_TF_IDF.append(1)
            else:
                predictions_TF_IDF.append(0)
            if correct2[real_len2-1] == 1.0:
                labels.append(1)
            else:
                labels.append(0)
            random_predictions.append(np.random.randint(low = 0, high=2))
    i+=1
    if i%1000==0:
        print(i)
print(predictions_TF_IDF)
print(labels)
print(random_predictions)
print(max_prob)

In [ ]:
problems_set2.save()

In [ ]:
def write_txt(file, data):
    with open(file, 'w') as file2:
        for d in data:
            file2.write(str(float(d)))
            file2.write(' ')
                    
write_txt(os.path.join(data_folder, 'predictions.txt'), predictions)

In [ ]:
acc = accuracy_score(labels, predictions_TF_IDF)
print(acc)
random_acc = accuracy_score(labels, random_predictions)
print(random_acc)